In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

### Prepare Data

In [2]:
# read data

df = pd.read_csv('all_task_environments_2020-10-14.csv', header=0)

### Define functions for prediction

In [3]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [4]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence,n=10):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,n):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [5]:
def get_avg_editdist(data, dimensions=[[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']],n=10,
                    seq='sequence', coords='coordinates', error='error'):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
        
    for row in range(0,len(data)):
        objects = list(data.at[row,seq])
        strong_k = list(data.at[row,'strong_k'].split(','))
        mid_k = list(data.at[row,'mid_k'].split(','))
        food_k = list(data.at[row,'food_k'].split(','))
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in data.at[row,coords].split(';'))}
        #print(coordinates)
        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        sequence = str(data.at[row,seq])
        
        for k2 in np.arange(1.1,2.0,0.1):
            k_food = round(k2,2)
            k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
            for k in np.arange(0,0.9,0.1):
                k_strong = round(k,2)
                k_mid = round(k + 0.1,2)
                k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                for c in np.arange(1.0,2.0,0.1):
                    c = round(c, 1)
                    c1 = {obj: c if obj in data.at[row, 'containment'] else 1.0 for obj in objects}
                
                    for dim in dimensions:                
                        # get average edit distance
                        median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence, n)
                        median = median / len(sequence)
                        params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                        results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
        results.at[row,'error'] = data.at[row,error]
        results.at[row,'ID'] = data.at[row,'ID']
                    
    return results

### Calculate edit distances, create df

In [6]:
# for n=10: 28min
%time results_new = get_avg_editdist(df)

CPU times: user 27min 47s, sys: 1 s, total: 27min 48s
Wall time: 27min 49s


In [55]:
results_new

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,0.400000,0.800000,0.800000,0.400000,0.200000,0.400000,0.000000,0.200000,0.600000,0.800000,...,0.200000,0.300000,0.500000,0.000000,0.300000,0.400000,0.200000,tnpsc,0.666,T1
1,0.400000,0.600000,0.800000,0.200000,0.300000,0.600000,0.200000,0.200000,0.600000,0.600000,...,0.200000,0.600000,0.400000,0.400000,0.400000,0.600000,0.400000,tnpcs,0.666,T14
2,0.200000,0.600000,0.600000,0.000000,0.200000,0.600000,0.000000,0.300000,0.600000,0.600000,...,0.200000,0.400000,0.400000,0.200000,0.300000,0.400000,0.200000,tnpsc,0.666,T16
3,0.400000,0.400000,0.400000,0.200000,0.400000,0.400000,0.200000,0.400000,0.400000,0.400000,...,0.400000,0.200000,0.200000,0.000000,0.000000,0.200000,0.000000,tnspc,0.666,T18
4,0.166667,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,...,0.000000,0.333333,0.333333,0.500000,0.000000,0.333333,0.333333,pms,0.440,E01_01
5,0.700000,0.600000,0.800000,0.800000,0.400000,0.200000,0.400000,0.800000,0.300000,0.800000,...,0.700000,0.400000,0.800000,0.400000,0.400000,0.800000,0.800000,rhcse,0.666,E01_03
6,1.000000,0.750000,0.875000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,1.000000,...,0.750000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,pbhs,0.605,E01_05
7,0.500000,1.000000,0.750000,1.000000,0.500000,1.000000,1.000000,1.000000,1.000000,0.625000,...,1.000000,1.000000,0.750000,1.000000,1.000000,1.000000,1.000000,hscp,0.605,E01_09
8,0.333333,0.333333,0.333333,0.000000,0.333333,0.666667,0.000000,0.000000,0.333333,0.500000,...,0.333333,0.333333,0.666667,0.000000,0.000000,0.666667,0.000000,pbs,0.440,E10_01
9,0.000000,0.333333,0.333333,0.333333,0.166667,0.333333,0.333333,0.000000,0.333333,0.000000,...,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,pgs,0.440,E12_01


### Get lowest error, compare edit distances

In [8]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [57]:
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx

(0.6315319184827748, Index(['c: 1.5; k: 0.2,0.3,1.5; xy'], dtype='object'))

In [10]:
np.nanmean(list_mean), np.nanmedian(list_mean)

(0.6948012558333564, 0.6958466024130581)

In [12]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.6678712375539545, stdev: 0.025122644902842033, median: 0.6565983666784113
Average xyz: 0.6677198684164155, stdev: 0.023842194574808257, median: 0.6559830361580175


In [13]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.7073210940254734, min: 0.6702247584879751
Average y: 0.7083795364624629, min: 0.6618528860896694
Average z: 0.7083254320713988, min: 0.6701281375611233
Average xz: 0.6963769620630297
Average yz: 0.7076146602407598


### Statistical analysis

In [14]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 1786.585, p =  0.000E+00


In [39]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_x, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 17298.000, p = 0.00000


### Plots for dataframe

In [19]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [27]:
%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median, alpha=0.4, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(results_mean['error'])
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.9,0.3,0.03,0.4]))
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: FixedFormatter should only be used together with FixedLocator


### Check for sequences worse than baseline

In [28]:
# get sequences from best model where performance worse than baseline
results_new['c: 1.5; k: 0.2,0.3,1.5; xy'].loc[results_new['c: 1.5; k: 0.2,0.3,1.5; xy'] >= np.mean(results_mean['error'])]

7     1.000000
15    0.888889
24    0.833333
30    0.785714
31    0.800000
32    0.833333
38    0.777778
39    0.875000
41    0.800000
42    0.777778
44    0.833333
46    0.785714
48    0.833333
51    0.812500
52    0.833333
55    0.777778
56    0.833333
57    0.833333
58    0.833333
60    0.785714
61    0.809524
62    0.777778
65    0.833333
68    0.875000
75    0.812500
76    0.857143
78    0.812500
Name: c: 1.5; k: 0.2,0.3,1.5; xy, dtype: float64

### Run model for only 2D/3D

In [29]:
df2 = pd.read_csv('all_task_environments_2020-10-14.csv', header=0)

In [37]:
# n=10: 8 min
# n=100: 1h 18
%time results_2D3D = get_avg_editdist(df2, dimensions=[[2,'xy'],[3,'xyz']],n=100)

CPU times: user 1h 17min 59s, sys: 770 ms, total: 1h 18min
Wall time: 1h 18min


In [59]:
results_2D3D

,"c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; xy","c: 1.1; k: 0.0,0.1,1.1; xyz","c: 1.2; k: 0.0,0.1,1.1; xy","c: 1.2; k: 0.0,0.1,1.1; xyz","c: 1.3; k: 0.0,0.1,1.1; xy","c: 1.3; k: 0.0,0.1,1.1; xyz","c: 1.4; k: 0.0,0.1,1.1; xy","c: 1.4; k: 0.0,0.1,1.1; xyz",...,"c: 1.6; k: 0.8,0.9,1.9; xyz","c: 1.7; k: 0.8,0.9,1.9; xy","c: 1.7; k: 0.8,0.9,1.9; xyz","c: 1.8; k: 0.8,0.9,1.9; xy","c: 1.8; k: 0.8,0.9,1.9; xyz","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.200000,0.000000,...,0.200000,0.000000,0.200000,0.000000,0.200000,0.000000,0.200000,tnpsc,0.666,T1
1,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,tnpcs,0.666,T14
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,tnpsc,0.666,T16
3,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,tnspc,0.666,T18
4,0.666667,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667,0.333333,pms,0.440,E01_01
5,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,...,0.800000,0.400000,0.800000,0.800000,0.800000,0.400000,0.800000,rhcse,0.666,E01_03
6,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,pbhs,0.605,E01_05
7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,hscp,0.605,E01_09
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pbs,0.440,E10_01
9,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,pgs,0.440,E12_01


In [61]:
lowest2, lowest_idx2, list_mean2, results_mean2 = get_lowest_error(results_2D3D)
lowest2, lowest_idx2

(0.631029607853882, Index(['c: 1.5; k: 0.3,0.4,1.1; xy'], dtype='object'))

In [42]:
#%matplotlib inline

error_individual = df['error']
ID = df['ID']
seqs = results_mean['sequence'][:-1].values
res = results_new['c: 1.3; k: 0.2,0.3,1.9; xy'][:-1].values
x = [x for x in range (0,len(seqs))]

plt.figure(figsize=(18,6))
plt.scatter(x, res, marker='o', label='model-generated sequences')
plt.scatter(x, error_individual, c='red', marker='o', linewidth=2, label='baseline for sequence length')
plt.plot(x, error_individual, c='red', alpha=0.5)
plt.plot((x,x),(error_individual,res), '--', c='black', alpha=0.6)


plt.title('best model (n=100), 0.626, c=1.3; k=0.2,0.3,1.9; xy', fontsize=16)
plt.xticks(x, ID, rotation=90)
plt.ylabel('median edit distance', fontsize=14)
plt.xlabel('sequence', fontsize=14)
plt.legend()

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

In [46]:
# Define input for 2D/3D plots

c2 = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k2 = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim2 = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median2 = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [49]:
# Plot scatterplots for xy, xyz only

%matplotlib qt

cm2 = ['blue','green']
dim_num2 = [0 if x=='xy' else 1 for x in dim]

cmap2 = matplotlib.colors.ListedColormap(cm2)

ticks2 = ['xy', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks2, cmap2.N)

# create figure, 3d grid, set background to white
fig3 = plt.figure(figsize=(12,8))
ax3 = fig3.add_subplot(111, projection='3d')
ax3.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img2 = ax3.scatter(c2, k2, median2, alpha=0.4, s=38, c=dim_num2, cmap=cmap2)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
zz = xx * 0 + np.mean(results_2D3D['error'])
ax3.plot_surface(xx, yy, zz, alpha=1.0)

# set labels
ax3.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax3.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax3.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)

cb = plt.colorbar(img2, cax = fig3.add_axes([0.9,0.3,0.03,0.1]))
cb.ax.set_yticklabels(ticks2, fontsize=16)

plt.show()

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: FixedFormatter should only be used together with FixedLocator


In [43]:
line = 'p: (0.008034,0.957082,0.6890539999999999);q: (-0.185,0.928,0.48103500000000005);c: (-0.525007,0.923283,0.438);g: (-0.595001,1.233,0.468999);k: (-0.030993,0.7334229999999999,0.530651);u: (-0.606068,0.743321,0.524829)'

In [44]:
test = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in line.split(';'))}